# Open double-funded channel without intermediary

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup LND clients for Ali and Bob

In [ ]:
import sys

In [ ]:
# LND GRPC bindings should be in '/home/jovyan/lnrpc'
sys.path.append('/home/jovyan/lnrpc')

In [ ]:
%%sh
sudo cp /ali-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/ali.macaroon
sudo chmod +r /tmp/ali.macaroon

sudo cp /bob-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/bob.macaroon
sudo chmod +r /tmp/bob.macaroon

In [ ]:
from p2oc.lnd_rpc import LndRpc, lnmsg, walletmsg, signrpc, signmsg, routermsg
from p2oc.signing import sign_inputs, create_change_only_psbt

In [ ]:
ali = LndRpc(host='ali-lnd:10009',
             cert_path='/ali-lnd/tls.cert',
             macaroon_path='/tmp/ali.macaroon')

In [ ]:
bob = LndRpc(host='bob-lnd:10009',
             cert_path='/bob-lnd/tls.cert',
             macaroon_path='/tmp/bob.macaroon')

In [ ]:
aliNodePubKey = ali.lnd.GetInfo(lnmsg.GetInfoRequest()).identity_pubkey
aliNodePubKey

In [ ]:
bobNodePubKey = bob.lnd.GetInfo(lnmsg.GetInfoRequest()).identity_pubkey
bobNodePubKey

## Fund Ali and Bob

In [ ]:
import os
import hashlib

import bitcoin
from p2oc.btc_rpc import Proxy, Config

In [ ]:
bitcoin.SelectParams('regtest')

In [ ]:
# only used to mine initial coins for Ali and Bob
brpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect='bitcoind',
    rpcport=18443
))

In [ ]:
brpc.getblockcount()

In [ ]:
aliAddr = ali.lnd.NewAddress(lnmsg.NewAddressRequest(type=0)) # p2wkh
aliAddr

In [ ]:
bobAddr = bob.lnd.NewAddress(lnmsg.NewAddressRequest(type=0)) # p2wkh
bobAddr

In [ ]:
# fund ali
_ = list(brpc.generatetoaddress(1, aliAddr.address))

In [ ]:
# fund bob
_ = list(brpc.generatetoaddress(1, bobAddr.address))

In [ ]:
try:
    brpc.createwallet('miner')
except:
    brpc.loadwallet('miner')

minerAddr = brpc.getnewaddress("coinbase")
brpc.unloadwallet('miner')

In [ ]:
# unlock mined coins
_ = list(brpc.generatetoaddress(110, minerAddr))

In [ ]:
ali.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

In [ ]:
bob.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

## Ali creates public offer to request inbound liquidity in exchange for a fee

In [ ]:
import time
import base64

from bitcointx.core.psbt import PartiallySignedTransaction, PSBT_ProprietaryTypeData, PSBT_Output
import bitcointx.core as bc
import bitcointx.core.script as bs
from bitcoin.rpc import hexlify, unhexlify
# XXX: bitcointx and bitcoin don't play nicely together

from p2oc.address import create_dummy_p2wpkh_address, next_key_desc
from p2oc.offer import Offer, attach_offer_to_psbt
from p2oc.funding import create_funding_output
from p2oc.channel import (
    connect_peer,
    generate_channel_id,
    create_channel_point_shim,
    register_channel_point_shim,
    open_channel
)

### Allocate funds

In [ ]:
# taker needs to pay premium to open channel
premiumAmount = int(0.001 * bc.CoreCoinParams.COIN) # premium Ali is willing to pay
fundAmount = int(0.16 * bc.CoreCoinParams.COIN) # requested inbound capacity

In [ ]:
psbt = create_change_only_psbt(premiumAmount, ali)

### Create offer

In [ ]:
aliKeyDesc = next_key_desc(ali)

offer = Offer(
    node_host='ali-lnd',
    node_pubkey=aliNodePubKey,
    premium_amount=premiumAmount,
    fund_amount=fundAmount,
    channel_pubkey=hexlify(aliKeyDesc.raw_key_bytes),
    input_indices=list(range(len(psbt.unsigned_tx.vin))),
    output_indices=list(range(len(psbt.unsigned_tx.vout))),
)

attach_offer_to_psbt(offer, psbt)

offer

In [ ]:
offer_psbt = psbt.to_base64()
del psbt # delete psbt so we don't accidentally leak state to the other party
offer_psbt

## Bob accepts offer and sends reply

### Check that the offer is still valid

In [ ]:
psbt1 = PartiallySignedTransaction.from_base64(offer_psbt)
psbt1

In [ ]:
offer = psbt1.proprietary_fields[b'offer'][0].value
offer = Offer.deserialize(offer)
offer

In [ ]:
# check that funding UTXOs has not been spent
# note we can't use `lnd.GetTransactions` since it only knows about our wallet's transactions
# it's probably safe to skip this step because blockchain will prevent from double spending
for vin in psbt1.unsigned_tx.vin:
    utxo = brpc.gettxout(vin.prevout)
    assert utxo is not None

In [ ]:
premiumAmount = offer.premium_amount # Ali's premium to Bob
fundAmount = offer.fund_amount # Bob funds this amount

In [ ]:
feesAmount = psbt1.get_fee() - premiumAmount
assert feesAmount > 0
feesAmount

### Allocate funds

In [ ]:
psbt2 = create_change_only_psbt(fundAmount, bob)

### Add our inputs and outputs to the offer psbt

In [ ]:
for i, vin in enumerate(psbt2.unsigned_tx.vin):
    inp = psbt2.inputs[i]
    inp.index = None # reset index
    psbt1.add_input(vin, inp)

In [ ]:
for i, vout in enumerate(psbt2.unsigned_tx.vout):
    out = psbt2.outputs[i]
    out.index = None # reset index
    psbt1.add_output(vout, out)

In [ ]:
del psbt2

### Create funding output

In [ ]:
bobKeyDesc = next_key_desc(bob)

In [ ]:
fundingOutput = create_funding_output(
    taker_pubkey=unhexlify(offer.channel_pubkey),
    maker_pubkey=bobKeyDesc.raw_key_bytes,
    premium_amount=premiumAmount,
    fund_amount=fundAmount
)

# add funding output to the original psbt must be the last one
psbt1.add_output(fundingOutput, PSBT_Output())

assert psbt1.get_output_amounts()[-1] == fundAmount + premiumAmount

In [ ]:
psbt1.get_input_amounts()

In [ ]:
# TODO: currently both parties pay fees. It probably makes sense that only Ali pays fees
psbt1.get_fee()

### connect to taker (Ali)

In [ ]:
# TODO: Do this as early as possible in the workflow
connect_peer(node_pubkey=offer.node_pubkey, node_host=offer.node_host, lnd=bob)

### Register shim to prepare for channel opening

In [ ]:
channelId = generate_channel_id(unhexlify(offer.channel_pubkey), bobKeyDesc.raw_key_bytes)
channelId

In [ ]:
channel_point_shim = create_channel_point_shim(
    channel_id=channelId,
    psbt=psbt1,
    premium_amount=premiumAmount,
    fund_amount=fundAmount,
    local_key_desc=bobKeyDesc,
    remote_pubkey=unhexlify(offer.channel_pubkey)
)

register_channel_point_shim(channel_point_shim, bob)

### Send Reply

In [ ]:
inputIndices = []
for i in range(len(psbt1.unsigned_tx.vin)):
    if i not in offer.input_indices:
        inputIndices.append(i)

In [ ]:
outputIndices = []
for i in range(len(psbt1.unsigned_tx.vout) - 1): # '-1' to exclude funding tx
    if i not in offer.output_indices:
        outputIndices.append(i)

In [ ]:
replyParams = {
    'nodeAddr': 'bob-lnd',
    'nodePubKey': bobNodePubKey,
    'channelId': channelId,
    'chanPubKey2': hexlify(bobKeyDesc.raw_key_bytes),
    'inputIndices': inputIndices,
    'outputIndices': outputIndices
}
replyParams

In [ ]:
import json

psbt1.proprietary_fields[b'reply'] = [
    PSBT_ProprietaryTypeData(0, b'params', json.dumps(replyParams).encode()),
    PSBT_ProprietaryTypeData(0, b'signature', b'TODO add signature')
]

In [ ]:
# add signature to offer reply
offerReply = psbt1.to_base64()
del psbt1
offerReply

## Ali checks offer reply and opens pending channel

In [ ]:
psbt = PartiallySignedTransaction.from_base64(offerReply)

In [ ]:
offer = psbt.proprietary_fields[b'offer'][0].value
offer = Offer.deserialize(offer)
offer

In [ ]:
replyParams = psbt.proprietary_fields[b'reply'][0].value
replyParams = json.loads(replyParams)
replyParams

### Check offer reply

In [ ]:
psbt.get_input_amounts()

In [ ]:
psbt.get_output_amounts()

In [ ]:
psbt.get_fee()

In [ ]:
premiumAmount = offer.premium_amount
fundAmount = offer.fund_amount

In [ ]:
# fundign output is the last one
assert premiumAmount + fundAmount == psbt.get_output_amounts()[-1]

In [ ]:
# TODO: check that our inputs and outputs were included

### Open pending channel

In [ ]:
channel_point_shim = create_channel_point_shim(
    channel_id=channelId,
    psbt=psbt,
    premium_amount=premiumAmount,
    fund_amount=fundAmount,
    local_key_desc=aliKeyDesc,
    remote_pubkey=unhexlify(replyParams['chanPubKey2'])
)

open_channel(
    node_pubkey=unhexlify(replyParams['nodePubKey']),
    fund_amount=fundAmount,
    premium_amount=premiumAmount,
    channel_point_shim=channel_point_shim,
    lnd=ali)

In [ ]:
# check that the channel is pending
ali.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

### Sign transaction

In [ ]:
# At this point we should have commitment transactions signed and we can sign the funding transaction
# TODO: how can we check with lnd that this is the case?

In [ ]:
# go through all of our inputs and try to sign them
sign_inputs(psbt, offer.input_indices, ali)

### Send pending channel reply

In [ ]:
pendingChannelReply = psbt.to_base64()
del psbt

## Bob checks pending channel reply, signs and commits funding tx

In [ ]:
psbt = PartiallySignedTransaction.from_base64(pendingChannelReply)

In [ ]:
offer = psbt.proprietary_fields[b'offer'][0].value
offer = Offer.deserialize(offer)
offer

In [ ]:
fundAmount = offer.fund_amount
premiumAmount = offer.premium_amount

In [ ]:
replyParams = psbt.proprietary_fields[b'reply'][0].value
replyParams = json.loads(replyParams)
replyParams

### Check that pending channel matches the offer

In [ ]:
# check that the channel is pending
resp = bob.lnd.PendingChannels(lnmsg.PendingChannelsRequest())
resp

In [ ]:
channel_point = f'{bc.b2lx(psbt.unsigned_tx.GetTxid())}:{len(psbt.unsigned_tx.vout)-1}'
channel_point

In [ ]:
target_channel = None

for pending_chan in resp.pending_open_channels:
    if pending_chan.channel.channel_point == channel_point:
        target_channel = pending_chan.channel
        break

In [ ]:
assert target_channel.local_balance == fundAmount
# TODO: check why remote_balance was slightly reduced. Looks like fees
# assert target_channel.remote_balance == premiumAmount
assert target_channel.remote_balance > 0

assert target_channel.remote_node_pub == offer.node_pubkey

### Sign funding tx

In [ ]:
sign_inputs(psbt, replyParams['inputIndices'], bob)

### Finalize and publish funding tx

In [ ]:
tx = bc.CMutableTransaction.from_instance(psbt.unsigned_tx)

In [ ]:
wits = map(lambda inp: inp.final_script_witness, psbt.inputs)
wits = list(map(bc.CTxInWitness, wits))

tx.wit = bc.CTxWitness(wits)

In [ ]:
bob.wallet.PublishTransaction(
    walletmsg.Transaction(tx_hex=tx.serialize()))

In [ ]:
_ = list(brpc.generatetoaddress(6, minerAddr))

### Check that the channel is active

In [ ]:
# should be no pending channels
bob.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

In [ ]:
bob.lnd.ListChannels(lnmsg.ListChannelsRequest())